In [17]:
import helper
from helper import *

# To plot pretty figures
%matplotlib widget

### Train a DNN on the modified dataset

In [18]:
# Get split returns a generator
# List comprehension is one way to evaluate a generator

original_data, modded_samples, training_labels, original_labels = simulate_blobs(class_size=5000)

# Separating a hold out set that will be used for validation later
X_train, y_train, X_test, y_test, y_original, X_valid, y_valid, y_valid_original = get_train_test_val(modded_samples, original_labels, training_labels)


print("Train Size:", X_train.shape)
print("Test Size:", y_test.shape)


hot_encoder = dfHotEncoder()
hot_encoder.fit(training_labels)
print("Categories:", hot_encoder.categories_)

FigureCanvasNbAgg()

Train Size: (4500, 2)
Test Size: (500,)
Categories: [array([0, 1])]


In [3]:
NUM_FEATURES = X_train.shape[1]
NUM_LABELS = len(hot_encoder.categories_[0])

In [39]:
def build_dnn(num_features, num_nodes = 16, depth = 2, num_labels=2, activation = "elu"):
    
    keras.backend.clear_session()

    nn = keras.models.Sequential()
    Dense = keras.layers.Dense
    
    # Using He initialization
    he_init = tf.keras.initializers.he_uniform()
    
    nn.add(Dense(units = num_nodes, activation=activation, input_dim=num_features,
                kernel_initializer=he_init))
    
    for i in range(1,depth):
        nn.add(Dense(units = num_nodes, activation=activation,
                    kernel_initializer=he_init))

    nn.add(Dense(units=num_labels, activation= "softmax",
                kernel_initializer=he_init))
    
    nn.compile(loss="categorical_crossentropy",
                  optimizer='sgd',
                  metrics=['accuracy'])
    
    return nn

def train_model(model, X, y, epochs=30, batch_size=20, verbose=0):
    
    ZScaler = StandardScaler().fit(X)
    
    X_train = ZScaler.transform(X)
    y_train = hot_encoder.transform(y)

    history = model.fit(X_train, y_train, epochs=epochs, batch_size = batch_size, verbose=verbose)
    
    return history, ZScaler


In [29]:
nn = build_dnn(NUM_FEATURES)
%time history, Zscaler = train_model(nn, X_train, y_train, X_test, y_test, epochs=50, batch_size=20)

Train on 4500 samples, validate on 500 samples
Epoch 1/50
4500/4500 [==============================] - 0s 66us/step - loss: 0.2082 - acc: 0.9687 - val_loss: 0.0801 - val_acc: 1.0000
Epoch 2/50
4500/4500 [==============================] - 0s 33us/step - loss: 0.0525 - acc: 1.0000 - val_loss: 0.0373 - val_acc: 1.0000
Epoch 3/50
4500/4500 [==============================] - 0s 32us/step - loss: 0.0281 - acc: 1.0000 - val_loss: 0.0232 - val_acc: 1.0000
Epoch 4/50
4500/4500 [==============================] - 0s 32us/step - loss: 0.0186 - acc: 1.0000 - val_loss: 0.0165 - val_acc: 1.0000
Epoch 5/50
4500/4500 [==============================] - 0s 33us/step - loss: 0.0137 - acc: 1.0000 - val_loss: 0.0126 - val_acc: 1.0000
Epoch 6/50
4500/4500 [==============================] - 0s 33us/step - loss: 0.0107 - acc: 1.0000 - val_loss: 0.0102 - val_acc: 1.0000
Epoch 7/50
4500/4500 [==============================] - 0s 34us/step - loss: 0.0087 - acc: 1.0000 - val_loss: 0.0085 - val_acc: 1.0000
Epoch 8/

In [30]:
# Plotting results from history
plot_history(history)

FigureCanvasNbAgg()

In [38]:
nn.evaluate(Zscaler.transform(X_test),hot_encoder.transform(y_test))

500/500 [==============================] - 0s 16us/step


[0.15590375757217406, 0.926]

## Performing LRP

In [31]:
import innvestigate
import innvestigate.utils as iutils

plot_args = {"kind":"scatter", "x":0,  "y":1, "c":"label", "cmap": "Set1", "s":10, "alpha":0.25}

def perform_analysis(model, analyzer, data, labels=[]):
    analysis = analyzer.analyze(data)
    prediction = model.predict(data)
    
    df_anal = pd.DataFrame(analysis)
    
    return df_anal


scaled_samples = Zscaler.transform(X_valid)

# Getting all the samples that can be correctly predicted
all_samples, _labels, correct_idxs = getCorrectPredictions(nn, scaled_samples, y_valid, enc = hot_encoder)
all_labels = y_valid_original[correct_idxs]


# Stripping the softmax activation from the model
model_w_softmax = nn
model = iutils.keras.graph.model_wo_softmax(model_w_softmax)

# Creating an analyzer
lrp_E = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPEpsilon(model=model, epsilon=1e-3)
# lrp_Z = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPZPlus(model=model)

all_lrp_E = perform_analysis(model,lrp_E, all_samples)

Prediction Accuracy
5000/5000 [==============================] - 0s 8us/step
Scores on data set: loss=0.001 accuracy=1.0000


In [32]:
plt.close("Positive Only LRP")
fig, axs = plt.subplots(1,3, figsize=(18,6), num="Positive Only LRP")

plot_args["c"] = "label"
original_data.plot(ax=axs[0], title="Original Distribution", **plot_args)

plot_args["c"] = all_labels
all_lrp_E.plot(ax=axs[1], title="LRP E", **plot_args)

pos_lrp = all_lrp_E.copy()
pos_lrp[pos_lrp<0] = 0
pos_lrp["label"] = all_labels.values
pos_lrp.plot(ax=axs[2],title="LRP E", **plot_args)

plt.tight_layout()
plt.show()

FigureCanvasNbAgg()

# Peforming an analysis on depths

In [28]:
depth_epochs = {2:50,3:50,4:100,5:100}


def runDNN(depth, epochs):
    nn = build_dnn(NUM_FEATURES)
    history, ZScaler = train_model(nn, X_train, y_train, 
                                   epochs=epochs, batch_size=20)
    
    
    scaled_samples = ZScaler.transform(X_valid)
    final_acc = nn.evaluate(scaled_samples,hot_encoder.transform(y_test))

    # Getting all the samples that can be correctly predicted
    all_samples, _labels, correct_idxs = getCorrectPredictions(nn, scaled_samples, y_valid, enc = hot_encoder)
    lrp_labels = y_valid_original[correct_idxs]


    # Stripping the softmax activation from the model
    model_w_softmax = nn
    model = iutils.keras.graph.model_wo_softmax(model_w_softmax)

    # Creating an analyzer
    lrp_E = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPEpsilon(model=model, epsilon=1e-3)
    lrp_results = perform_analysis(model,lrp_E, all_samples)
    
    return (final_acc, lrp_results, lrp_labels)

plt.close("Depth Comparison")
fig, axs = plt.subplots(len(depths),1, figsize=(15,8*len(depths)), num="Depth Comparison")
plt.tight_layout()

_labels = all_labels.values

# depth_epochs = {2:50}

# results = runDNN()

for i,d in enumerate(depth_epochs):
    
    final_acc, 
    
    pos_lrp = lrp_results.copy()
    pos_lrp[pos_lrp<0] = 0
    data = pos_lrp.values

    axs[i].scatter(*data.T, s=50, linewidth=0, c=lrp_labels, alpha=0.5, cmap=plot_args["cmap"])
    axs[i].set_title("Depth: {}".format(d))


# pos_lrp["label"] = all_labels.values
# pos_lrp.plot(ax=axs[2],title="LRP E", **plot_args)

FigureCanvasNbAgg()